In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
import datetime

In [2]:
filename = '../inputs/Maltas.csv'
df = pd.read_csv(filename)
df =df[df['Quantity'] > 0]

#Test train for next day
new_matrix_test = df.drop_duplicates(['Account_id',	'Product_id', 'Category'])
new_matrix_test['Date'] = (pd.to_datetime(df.Date.max()) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



C:\Users\willy\AppData\Local\Temp\ipykernel_9480\3843012267.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_matrix_test['Date'] = (pd.to_datetime(df.Date.max()) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')


In [3]:
train, test = train_test_split(df, test_size=0.30)
train = train.sort_values('Date')
test = test.sort_values('Date')

In [4]:
train.shape

(14117331, 5)

In [5]:
test.shape

(6050285, 5)

In [6]:
def date_features(df: pd.DataFrame) -> pd.DataFrame:
    """Import the DataFrame to be able to extract the data characteristics and transform

    Arguments:
        df: data frame

    Return:
        Return a DataFrame with its date characteristics
    """

    df['date'] = pd.to_datetime(df['Date'])
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    df['dayofyear'] = df.date.dt.dayofyear
    df['dayofweek'] = df.date.dt.dayofweek
    df['weekofyear'] = df.date.dt.isocalendar().week
    
    # Additionnal Data Features
    df['day^year'] = np.log((np.log(df['dayofyear'] + 1)) ** (df['year'] - 2000))
    
    # Drop date
    df.drop('Date', axis=1, inplace=True)
    
    return df

# Dates Features for Train, Test
train = date_features(train)
test = date_features(test)

In [7]:
train.head()

,Account_id,Product_id,Category,Quantity,date,year,month,day,dayofyear,dayofweek,weekofyear,day^year
10560,33231588,15354,Maltas,3,2022-01-03,2022,1,3,3,0,1,7.185954
9434,33228849,8260,Maltas,3,2022-01-03,2022,1,3,3,0,1,7.185954
28033,33226443,8260,Maltas,1,2022-01-03,2022,1,3,3,0,1,7.185954
8926,33231651,11910,Maltas,1,2022-01-03,2022,1,3,3,0,1,7.185954
5640,33251433,11910,Maltas,4,2022-01-03,2022,1,3,3,0,1,7.185954


In [8]:
test.head()

,Account_id,Product_id,Category,Quantity,date,year,month,day,dayofyear,dayofweek,weekofyear,day^year
19523,36469677,11910,Maltas,1,2022-01-03,2022,1,3,3,0,1,7.185954
21139,33228750,8260,Maltas,5,2022-01-03,2022,1,3,3,0,1,7.185954
28945,33746682,8262,Maltas,1,2022-01-03,2022,1,3,3,0,1,7.185954
15359,33221244,8262,Maltas,1,2022-01-03,2022,1,3,3,0,1,7.185954
15629,33224598,15354,Maltas,10,2022-01-03,2022,1,3,3,0,1,7.185954


In [9]:
# Daily Average, Monthly Average for train
train['daymonth_avg']  = train.groupby(['Product_id','Account_id', 'dayofweek'])['Quantity'].transform('mean')
train['monthly_avg'] = train.groupby(['Product_id','Account_id','month'])['Quantity'].transform('mean')
train = train.dropna()

# Average sales for Day_of_week = d per 'Category', 'Account_id'
daymonth_avg = train.groupby(['Product_id','Account_id','dayofweek'])['Quantity'].mean().reset_index()

# Average sales for Month = m per 'Category','Account_id' 
monthly_avg = train.groupby(['Product_id','Account_id','month'])['Quantity'].mean().reset_index()



# Merge Test with Daily Avg, Monthly Avg
def merge(df1: pd.DataFrame, df2: pd.DataFrame, col: list, col_name: str) -> pd.DataFrame:
    """This is function to do merge whit DataFrames and strings what representation a new DataFrame

    Args: 
        df1: pd.DataFrame is a df of test
        df2: pd.DataFrame is a df of average sales for day of week or month
        col: list with features of test
        col_name: str with name of column per feature of week or month 
    Return: 
          The return value is a pd.DataFrame with features news
    """
    
    df1 =pd.merge(df1, 
                  df2, 
                  how='left', 
                  on=None, 
                  left_on=col, 
                  right_on=col,
                  left_index=False, 
                  right_index=False, 
                  sort=True,
                  copy=True, 
                  indicator=False)
    
    df1 = df1.rename(columns={'sales':col_name})
    return df1


In [10]:
# Add Daily_avg and Monthly_avg features to test 
test = merge(test, daymonth_avg, ['Product_id','Account_id','dayofweek'],'daymonth_avg')
test = merge(test, monthly_avg, ['Product_id','Account_id','month'],'monthly_avg')

In [11]:
# Sales Rolling mean sequence per item 

rolling_10 = train.groupby(['Product_id'])['Quantity'].rolling(3).mean().reset_index().drop('level_1', axis=1)
train['rolling_mean'] = rolling_10['Quantity'] 

In [12]:

rolling_10.head()

,Product_id,Quantity
0,8260,NaN
1,8260,NaN
2,8260,1.666667
3,8260,5.666667
4,8260,5.666667


In [13]:
# 90 last days of training rolling mean sequence added to test data
rolling_last90 = train.groupby(['Product_id','Quantity'])['rolling_mean'].tail(90).copy()
test['rolling_mean'] = rolling_last90.reset_index().drop('index', axis=1)

In [14]:
rolling_last90.head()

1352     13.666667
27128     1.666667
10757     1.000000
2258      8.000000
20703     4.000000
Name: rolling_mean, dtype: float64

In [15]:

# Shifting rolling mean 3 months
train['rolling_mean'] = train.groupby(['Product_id'])['rolling_mean'].shift(90) # Create a feature with rolling mean of day - 90

In [16]:
test.head()

,Account_id,Product_id,Category,Quantity_x,date,year,month,day,dayofyear,dayofweek,weekofyear,day^year,Quantity_y,Quantity,rolling_mean
0,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,13.666667
1,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,1.666667
2,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,1.000000
3,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,8.000000
4,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,4.000000


In [17]:
test = test.fillna(0)
test.head()

,Account_id,Product_id,Category,Quantity_x,date,year,month,day,dayofyear,dayofweek,weekofyear,day^year,Quantity_y,Quantity,rolling_mean
0,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,13.666667
1,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,1.666667
2,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,1.000000
3,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,8.000000
4,33217773,8260,Maltas,20,2022-01-04,2022,1,4,4,1,1,10.46947,20.778626,18.375635,4.000000


In [18]:
# Clean features highly correlated to each others
for df in [train, test]:
    df.drop(['dayofyear', 
             'weekofyear',
             'day',
             'month',
             'Category',
             'Account_id'],
                axis=1, 
                inplace=True)

In [19]:
# Features Scaling (except sales)
sales_series, id_series = train['Quantity'], test['Product_id']

In [20]:
# Retrieve actual Sales values and ID
train['Quantity'] = sales_series
test['Product_id'] = id_series

In [21]:
# Training Data
X_train = train.drop('Quantity', axis=1).dropna()
y_train = train['Quantity']

In [22]:
# Test Data
test.sort_values(by=['Product_id'], inplace=True)
X_test = test.drop('Product_id', axis=1)

In [23]:
#df = train
df_train = train.copy()


In [24]:
df_train = df_train.drop(columns='date').fillna(0)

In [25]:
# Train Test Split
X_train , X_test ,y_train, y_test = train_test_split(df_train.drop('Quantity',axis=1),df_train.pop('Quantity'), random_state=123, test_size=0.2)

In [26]:
X_test

,Product_id,year,dayofweek,day^year,daymonth_avg,monthly_avg,rolling_mean
1427316,15354,2022,4,21.917036,1.000000,1.000000,3.000000
19164099,8262,2022,2,36.577186,2.848485,1.000000,0.000000
2425485,11910,2022,5,25.141310,1.000000,1.000000,4.000000
10965619,15354,2022,3,33.829808,19.187817,11.996212,8.333333
17130428,15354,2022,4,35.971890,2.476658,3.464789,0.000000
...,...,...,...,...,...,...,...
1318639,8262,2022,4,21.917036,1.000000,1.000000,5.333333
452240,11910,2022,3,14.646056,1.147982,1.000000,1.666667
16532753,8262,2022,3,35.762929,1.000000,1.000000,0.000000
18441568,15354,2022,5,36.333589,1.500000,1.780952,0.000000


In [27]:
y_test

1427316      1
19164099     1
2425485      1
10965619    20
17130428     3
            ..
1318639      1
452240       1
16532753     1
18441568     1
17660823     7
Name: Quantity, Length: 2823467, dtype: int64

In [28]:
# XGB Model
matrix_train = xgb.DMatrix(X_train, label = y_train)
matrix_test = xgb.DMatrix(X_test, label = y_test)

In [29]:

# Run XGB 
model = xgb.train(params={'objective':'reg:linear','eval_metric':'mae'}
                ,dtrain = matrix_train, num_boost_round = 500, 
                early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

[22:42:15] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-mae:6.73420
[1]	test-mae:5.33942
[2]	test-mae:4.56411
[3]	test-mae:4.15424
[4]	test-mae:3.89985
[5]	test-mae:3.76161
[6]	test-mae:3.68884
[7]	test-mae:3.62409
[8]	test-mae:3.58514
[9]	test-mae:3.56709
[10]	test-mae:3.54640
[11]	test-mae:3.52378
[12]	test-mae:3.52199
[13]	test-mae:3.51631
[14]	test-mae:3.51376
[15]	test-mae:3.51146
[16]	test-mae:3.51025
[17]	test-mae:3.51295
[18]	test-mae:3.48988
[19]	test-mae:3.48742
[20]	test-mae:3.48100
[21]	test-mae:3.48176
[22]	test-mae:3.46217
[23]	test-mae:3.46127
[24]	test-mae:3.45398
[25]	test-mae:3.44356
[26]	test-mae:3.44382
[27]	test-mae:3.44151
[28]	test-mae:3.43693
[29]	test-mae:3.42838
[30]	test-mae:3.42287
[31]	test-mae:3.42203
[32]	test-mae:3.41477
[33]	test-mae:3.40672
[34]	test-mae:3.39516
[35]	test-mae:3.39054
[36]	test-mae:3.38663
[37]	test-mae:3.375

In [30]:
model.predict(matrix_test)

array([1.1344357 , 0.9684726 , 0.98501974, ..., 0.93712646, 1.8562307 ,
       6.1198993 ], dtype=float32)